<a href="https://colab.research.google.com/github/btshow/RNN_text_generator_from_scratch/blob/main/RNN_Text_Generation_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import os

data_path = '/your/data/path/to/dinos.txt'

In [ ]:
def initialize_parameters(n_a, n_x, n_y):
    Wax = np.random.randn(n_a, n_x)*0.01  # input to hidden weight matrix,  default: n_a x  n_x = n_a x vocab_size = 50 x 27
    #print(f'The dimensions of initialized weight matrix Wax are {Wax.shape}')
    Waa = np.random.randn(n_a, n_a)*0.01  # hidden to hidden weight matrix, default: n_a x  n_x = 50 x 50
    #print(f'The dimensions of initialized weight matrix Waa are {Waa.shape}')
    Wya = np.random.randn(n_y, n_a)*0.01  # hidden to output weight matrix, default: n_y x  n_a = vocab_size x n_a = 50 x 27
    #print(f'The dimensions of initialized weight matrix Wya are {Wya.shape}')
    ba = np.zeros((n_a, 1))  # hidden biases vector, default: ba =  n_a x 1 = 50 x 1
    #print(f'The dimensions of initialized bias vector ba are {ba.shape}')
    by = np.zeros((n_y, 1))  # output biases vector,  default: ba =  n_a x 1 = 27 x 1
    #print(f'The dimensions of initialized bias vector by are {by.shape}')

    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by} # collect parameters into a dictionary to have them all in one clear place
    return parameters
    
def rnn_cell_forward(xt, a_prev, parameters):
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)
    yt_pred = softmax(np.dot(Wya, a_next) + by)
    cache = (a_next, a_prev, xt, parameters)
    return a_next, yt_pred, cache


def rnn_forward(X, Y, a0, parameters, vocab_size=27):
    ''' 
    Performs the forward pass through the full RNN and also computes the cross-entropy loss.
    It returns the loss' value as well as a "cache" storing values to be used in backpropagation.
    '''
    x, a, y_hat = {}, {}, {}
    a[-1] = np.copy(a0)
    loss = 0

    # Encode the elements of the input sequence as dummy vectors: 
    # The elements of the input vector X are integers corresponding to the 
    # indices of the tokens in the vocabulary
    for t in range(len(X)):
        x[t] = np.zeros((vocab_size, 1))
        if (X[t] != None):
            x[t][X[t]] = 1
        
        # Compute the outputs a[t] and y_hat[t] of the RNN Cell
        a[t], y_hat[t], _ = rnn_cell_forward(x[t], a[t-1],parameters)
        
        # This computes the loss for a single time step
        # Due to the for loop the loss is accumulated for the whole output sequence
        loss -= np.log(y_hat[t][Y[t], 0])
    cache = (y_hat, a, x)
    return loss, cache




def rnn_cell_backward(dy, gradients, parameters, x, a, a_prev):
    # Compute the gradients of a given RNN Cell w.r.t. Wya, by, etc.
    # The gradients for an RNN Cell depend on quite a lot: 
    # - dy: deviation between predictions and true values 
    # - parameters: the parameters used to compute the predictions
    # - x: the vectors of the input sequence
    # - a: the activations computed in this cell
    # - a_prev: the activations passed from the previous cell 

    # Note: By using the += operator here, the gradients of all RNN cells 
    # are accumulated during the for loop in rnn_backward()

    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) +         gradients['da_next']  # backprop into h
    daraw = (1 - a * a) * da  # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients


def rnn_backward(X, Y, parameters, cache):
    """ Performs the backward propagation through time to compute the gradients of the loss with respect
    to the parameters. It returns also all the hidden states."""

    gradients = {}
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(ba), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])

    for t in reversed(range(len(X))):     
        dy = np.copy(y_hat[t])

        # Compute the deviation between prediction and true value 
        dy[Y[t]] -= 1

        # Compute the gradients needed for parameter updating
        # Due to the for loop gradients for all RNN cells are accumulated into gradients
        gradients = rnn_cell_backward(
            dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a





def update_parameters(parameters, gradients, lr):
    '''
    This function adjusts the parameters during backpropagation 
    such that in the next iteration (hopefully) a lower loss function value is achieved.
    A learning rate below 1 (e.g. lr = 0.01) ensures that parameters are not adjusted 
    too much with respect to the gradients calculated from the samples in the current batch.

    Note: The parameters are shared across the whole RNN. Therefore this function 
    updates the parameters only once for each training example. (See use of update_parameters() in optimize())
    '''
    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['ba'] += -lr * gradients['db']
    parameters['by'] += -lr * gradients['dby']
    
    return parameters


def softmax(x):
  '''
  Used as the output layer for each step of the RNN.  
  With 27 tokens in the vocabulary the output vector of softmax has 27 elements.
  '''
  e_x = np.exp(x - np.max(x)) 
  return e_x / e_x.sum(axis=0)




def sample(parameters, char_to_ix, seed):
    '''
    This uses the RNN Cell with computes the vocabulary
    '''
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]    

    # Use a vector of zeros for x<1> and a<0> as an input to the first RNN Cell 
    # to generate the first letter of the generated sequence
    x = np.zeros((vocab_size, 1)) 
    a_prev = np.zeros((n_a, 1))
    indices = []
    idx = -1 
    counter = 0
    newline_character = char_to_ix['\n']  
    
    #Iteratively generate and append letters i.e. token (more precicely the corresponding integers) 
    # using the RNN. Stop only when the generated token is the EOS token '\n' or
    # if the output sequence becomes longer than 50 letters.

    while (idx != newline_character and counter != 50):  
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + ba)
        z = np.dot(Wya, a) + by
        y = softmax(z)

        # randomly sample an integer (corresponding to a letter) using the probabilities in the output vector y    
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel()) 

        # Append the integer to our output sequence
        indices.append(idx)
        
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        a_prev = a
        
        counter +=1
        
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    '''
    This executes one step of the optimization to train the model. Particularly, 
    one step here means that one training examples is shown to the model and 
    parameters are updated w.r.t. this training example. 
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    '''
    # Forward pass and calculation of the loss in this iteration
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Retrieve gradients of the loss function wrt the parameters
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    gradients = clip(gradients, 5)   # The clip function supports faster training. It can in principle be left out.
    
    # Update the RNN parameters
    parameters = update_parameters(parameters, gradients, learning_rate)
        
    return loss, gradients, a[len(X)-1]


def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    '''
    This function trains an RNN text generation model. 
    As a side effect the function also prints every 2000 parameter upgrades 
    a bunch of generated dino name examples to get some intuition about how well 
    the model already captures the structure of dino names. 

    Inputs:
      data: string of dino names separated by "\n" (=newline symbol) 
      ix_to_char: dictionary to lookup the token(letter) corresponding to its numerical value
      char_to_ix: dictionary to lookup the numerical value correspondiong to its token(letter)
      num_iterations: Number of iterations to do backpropagation and parameter updates
      n_a: Number of activations in each RNN Cell
      dino_names: number of generated dino names after every 2000th iteration
      vocab_size: number of tokens in the vocabulary

    Output: 
    This function returns a dictionary containing the parameters of the fully trained model.
    '''
    n_x, n_y = vocab_size, vocab_size  # Each time step takes a character as input and outputs a character. The number of characters considered is determined by vocab_size.
    
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    loss = get_initial_loss(vocab_size, dino_names)
    
    with open(data_path, 'r')  as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    np.random.seed(0)
    np.random.shuffle(examples)
    
    a_prev = np.zeros((n_a, 1))
    
    # This loop runs through all training examples. 
    # If num_iterations is set higher than thee number of training examples the loop 
    # runs mulitple time through the training examples.
    for j in range(num_iterations):
        index = j % len(examples)
        # Encode training examples as vocabulary indices: 
        # Get vocabulary indices of the tokens for each example input sequence (X)
        # and output sequence (Y), where Y is the same as the input sequence X but 
        # shifted to the right by one and ending with the [EOS]-token "\n"
        # Note: The encoding of index numbers as dummy vectors happens inside the 
        # optimize function, more precisely inside the rnn_forward function that is 
        # called in the optimize function.
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]

        # Perform one optimization step 
        # This includes: Forward-prop -> Backward-prop -> Gradient Clipping -> Update Parameters
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters)
        
        # The loss smoothing is used to display a smoother loss evolution
        loss = smooth(loss, curr_loss)
        
        # Every 2000th Iteration, generate "n" characters thanks to sample() to 
        # check if the model is learning the structure of dino names properly
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            seed = 0
            for name in range(dino_names):
                
                # Use the sample function to generate one output sequence
                sampled_indices = sample(parameters, char_to_ix, seed)

                # Transform the vocabulary indices in sampled_indices to print text
                txt = ''.join(ix_to_char[ix] for ix in sampled_indices)
                txt = txt[0].upper() + txt[1:]  # capitalize first character
                print('%s' % (txt, ), end='')
                
                # change random seed to ensure sampling different output sequences 
                # each time a dino name is generated
                seed += 1  
      
            print('\n')
        
    return parameters


################################################################################
# Some tricks that help faster training, but are not essential components of an RNN
################################################################################

def smooth(loss, cur_loss):
  '''
  Loss smoothing is not an essential component for an RNN, but it helps to 
  accelerate the training. I use loss smoothing mainly to make the training 
  fast enough to show the full training run conveniently during the lecture.
  '''
  return loss * 0.999 + cur_loss * 0.001

def get_initial_loss(vocab_size, seq_length):
  '''
  This generates a starting value for the loss. 
  A starting value is needed to conveniently use loss smoothing. 
  The formula used ensures a reasonable start values for loss. 
  As with loss smoothing this is not an essential component for an RNN, 
  but it helps to accelerate the training. I use loss smoothing mainly to make the training 
  fast enough to show the full training run conveniently during the lecture.
  '''
  return -np.log(1.0/vocab_size)*seq_length

def clip(gradients, maxValue):
    ''' Gradient Clipping:
        Gradient clipping is not an essential component for an RNN, but a general way to 
        support a more stable training process for neural networks in general 
        (stable training process = the loss function does not wildly jump up and down with each batch).
        Gradient clipping is used mainly to make the training fast enough to show 
        the full training run conveniently during the lecture.

        Background: Sometimes extremely high or low gradients occur during a training iteration. 
        These high gradients would have a lot of "changing power" on the parameters 
        during the parameter updates and the loss function, correspondingly.
        In order to avoid too strong parameter updates, the gradients are clipped 
        i.e. restrained to stay in a certain range, here the range -maxValue, maxValue 
        Only after gradients are clipped the parameter updates are performed      
    '''
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients


def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character
    print('%s' % (txt, ), end='')

#########################################

# Load training data (lots of dinosaur names eparated by "\n" i.e. our [EOS] token
data = open(data_path, 'r').read()

# Transform training dino names to lower case (to keep the vocabulary at 27 parameters)
data= data.lower() 

# Get the set of all unique tokens
chars = list(set(data))

data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }

# Now train the model, after each 2000th iteration a bunch of dino names is generated
# to get an intuition of how well the RNN already captures the structure of dino names
# Finally the parameters of the trained model are stored in trained_parameters
trained_parameters = model(data, ix_to_char, char_to_ix)